In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_key import g_key

In [8]:
# Export cities csv
cities = "../../output_data/cities.csv"
cities_df = pd.read_csv(cities)
cities_df

,Unnamed: 0,City,Max Temp,Cloudiness,Humidity,Wind Speed,Country,Latitude,Longitude,Date
0,0,Punta Arenas,46.40,75,66,21.85,CL,-53.1500,-70.9167,1611788886
1,1,Kamaishi,33.51,100,83,4.32,JP,39.2667,141.8833,1611788887
2,2,Chumikan,-13.14,100,88,8.72,RU,54.7183,135.3150,1611788888
3,3,Henties Bay,68.00,6,85,6.13,NaN,-22.1160,14.2845,1611788889
4,4,Pangnirtung,-4.00,1,77,4.61,CA,66.1451,-65.7125,1611788890
...,...,...,...,...,...,...,...,...,...,...
558,558,Kassala,70.88,61,58,5.95,SD,15.4510,36.4000,1611789526
559,559,Jiangdu,42.01,18,94,14.85,CN,32.4286,119.5574,1611789527
560,560,Lompoc,53.60,90,87,14.97,US,34.6391,-120.4579,1611789325
561,561,Honāvar,70.32,2,71,3.56,IN,14.2833,74.4500,1611789529


In [55]:
# Create gmaps figure that shows the locations of the cities
gmaps.configure(api_key=g_key)
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df["Humidity"]
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=3,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

plt.savefig("../../output_data/heatmap1.png")

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [42]:
# Narrow down cities to ideal weather conditions
narrowed1_df = cities_df.loc[(cities_df["Max Temp"] > 60) & (cities_df["Max Temp"] <= 80)]
narrowed2_df = narrowed1_df.loc[cities_df["Humidity"] <= 20]
narrowed3_df = narrowed2_df.loc[cities_df["Wind Speed"] < 10]

narrowed3_df

,Unnamed: 0,City,Max Temp,Cloudiness,Humidity,Wind Speed,Country,Latitude,Longitude,Date
210,210,Diffa,70.16,0,18,9.60,NE,13.3154,12.6113,1611789128
269,269,Am Timan,73.51,68,20,6.44,TD,11.0297,20.2827,1611789195
331,331,Nombre de Dios,71.60,40,5,9.22,MX,23.8500,-104.2333,1611789264
464,464,Natitingou,75.85,2,17,6.49,BJ,10.3042,1.3796,1611789418


In [57]:
hotel_df = narrowed3_df

# Parameters dictionary
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Find closest hotels
for index, row in hotel_df.iterrows():
    
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    params["location"] = f"{latitude},{longitude}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        pd.options.mode.chained_assignment = None
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
# Unsure why an error is occurring during the try

Retrieving Results for Index 210: Diffa.
Closest hotel is ProDAF/Diffa.
------------
Retrieving Results for Index 269: Am Timan.
Closest hotel is Palais du chef Canton Salamt.
------------
Retrieving Results for Index 331: Nombre de Dios.
Closest hotel is Uraján de Luna Hotel Boutique & SPA.
------------
Retrieving Results for Index 464: Natitingou.
Closest hotel is Auberge Centrale.
------------


In [46]:
hotel_df

,Unnamed: 0,City,Max Temp,Cloudiness,Humidity,Wind Speed,Country,Latitude,Longitude,Date,Hotel Name
210,210,Diffa,70.16,0,18,9.60,NE,13.3154,12.6113,1611789128,ProDAF/Diffa
269,269,Am Timan,73.51,68,20,6.44,TD,11.0297,20.2827,1611789195,Palais du chef Canton Salamt
331,331,Nombre de Dios,71.60,40,5,9.22,MX,23.8500,-104.2333,1611789264,Uraján de Luna Hotel Boutique & SPA
464,464,Natitingou,75.85,2,17,6.49,BJ,10.3042,1.3796,1611789418,Auberge Centrale


In [56]:
# Create gmap to show the locations of the hotels
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
plt.savefig("../../output_data/heatmap2.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>